In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from utils.udp_util import extract_udp_prague_to_dataframe
from utils.plotter import  plot_matplotlib_graph, plot_plotly_graph, plot_cdf_matplotlib, plot_cdf_plotly, plot_boxplot_seaborn
from utils.util import extract_iperf3_data_df, find_files_by_folder
from datetime import datetime

In [ ]:
import os



# Usage example
root_folder = "../data"
file_extensions = [".json"]  # Add whatever extensions you want

files_dict = find_files_by_folder(root_folder, file_extensions)
print(files_dict)


In [ ]:
files_dict.keys()

In [ ]:
from utils.util import create_directory_if_not_exists
import os
graph_directory = os.path.join("./", "Graphs")
create_directory_if_not_exists(graph_directory)

In [ ]:
def extract_latest_by_type(files_dict):
    best = None
    best_time = None
    baseline = None
    baseline_time = None

    for key in files_dict:
        # Extract timestamp and type
        *_, timestamp_str, tag = key.split('_')
        timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d-%H-%M-%S")

        if tag == 'best' and (best_time is None or timestamp > best_time):
            best_time = timestamp
            best = key
        elif tag == 'baseline' and (baseline_time is None or timestamp > baseline_time):
            baseline_time = timestamp
            baseline = key

    return {
        'DRL_allocation': files_dict[best] if best else {},
        'Random_allocation': files_dict[baseline] if baseline else {}
    }

# Usage
latest_folders = extract_latest_by_type(files_dict)

In [ ]:
latest_folders

In [ ]:
latest_folders['DRL_allocation']

In [ ]:
latest_folders['Random_allocation']

In [ ]:
tfcubic = extract_iperf3_data_df(latest_folders['DRL_allocation'][0])
utf = extract_udp_prague_to_dataframe(latest_folders['DRL_allocation'][2])

In [ ]:
# import pandas as pd

# # Assuming you already have tfcubic loaded
# q1 = tfcubic['SmoothedRTT'].quantile(0.25)
# q3 = tfcubic['SmoothedRTT'].quantile(0.75)
# iqr = q3 - q1

# # Define bounds for non-outliers
# lower_bound = q1 - 1.5 * iqr
# upper_bound = q3 + 1.5 * iqr

# # Filter out outliers
# tfcubic_cleaned = tfcubic[(tfcubic['SmoothedRTT'] >= lower_bound) & (tfcubic['SmoothedRTT'] <= upper_bound)]


In [ ]:
# Define a function to remove outliers based on IQR for a given column
def remove_outliers_iqr(df, columns=['SmoothedRTT', 'thrpt']):

    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Filter rows where values are within the bounds
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]   

    return df

In [ ]:
# paths = {
#     "Cubic - DRL_allocation ":extract_iperf3_data_df(latest_folders['DRL_allocation'][0]),
#     "L4S - DRL_allocation ":extract_udp_prague_to_dataframe(latest_folders['DRL_allocation'][2]),
#     "Cubic - Random_allocation ":extract_iperf3_data_df(latest_folders['Random_allocation'][0]),
#     "L4S - Random_allocation ":extract_udp_prague_to_dataframe(latest_folders['Random_allocation'][2]),
# }


# paths = {
#     "Cubic - DRL_allocation ":remove_outliers_iqr(extract_iperf3_data_df(latest_folders['DRL_allocation'][0])),
#     "L4S - DRL_allocation ":remove_outliers_iqr(extract_udp_prague_to_dataframe(latest_folders['DRL_allocation'][2])),
#     "Cubic - Random_allocation ":remove_outliers_iqr(extract_iperf3_data_df(latest_folders['Random_allocation'][0])),
#     "L4S - Random_allocation ":remove_outliers_iqr(extract_udp_prague_to_dataframe(latest_folders['Random_allocation'][2])),
# }



paths = {
    "Cubic–DRL":extract_iperf3_data_df(latest_folders['DRL_allocation'][0]),
    "L4S–DRL":extract_udp_prague_to_dataframe(latest_folders['DRL_allocation'][2]),
    "Cubic–Rand":extract_iperf3_data_df(latest_folders['Random_allocation'][0]),
    "L4S–Rand":extract_udp_prague_to_dataframe(latest_folders['Random_allocation'][2]),
}


paths = {
    "Cubic–DRL":remove_outliers_iqr(extract_iperf3_data_df(latest_folders['DRL_allocation'][0])),
    "L4S–DRL":remove_outliers_iqr(extract_udp_prague_to_dataframe(latest_folders['DRL_allocation'][2])),
    "Cubic–Rand":remove_outliers_iqr(extract_iperf3_data_df(latest_folders['Random_allocation'][0])),
    "L4S–Rand":remove_outliers_iqr(extract_udp_prague_to_dataframe(latest_folders['Random_allocation'][2])),
}




# Matplotlib Time-Series

In [ ]:
scenario = str("RTT_UDPPraguevsCubic")
plot_matplotlib_graph(paths=paths,
                ycolumn="SmoothedRTT",
                title=f"{scenario} SmoothedRTT",
                xlabel="Time (s)",
                ylabel="SmoothedRTT (ms)",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)

scenario = str("Thrpt_UDPPraguevsCubic")
plot_matplotlib_graph(paths=paths,
                ycolumn="thrpt",
                title=f"{scenario} Throuhghput",
                xlabel="Time (s)",
                ylabel="Throuhghput (mbps)",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)


scenario = str("PktLoss_UDPPraguevsCubic")
plot_matplotlib_graph(paths=paths,
                ycolumn="Lost_Packets",
                title=f"{scenario} Packet Loss",
                xlabel="Time (s)",
                ylabel="Packet Loss",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)

# Matplotlib CDF

In [ ]:
# Convert paths dict of DataFrames to dict of arrays for CDF plot
def get_cdf_data(paths, ycolumn):
    cdf_data = {}
    for label, df in paths.items():
        cdf_data[label] = df[ycolumn].dropna().values
    return cdf_data


scenario = "RTT_UDPPraguevsCubic"
cdf_data = get_cdf_data(paths, "SmoothedRTT")
plot_cdf_matplotlib(
    data_dict=cdf_data,
    xlabel="SmoothedRTT (ms)",
    ylabel="CDF",
    title=f"{scenario} SmoothedRTT CDF",
    filename=f"{scenario}_SmoothedRTT_CDF",
    folder=graph_directory,
)

scenario = "Thrpt_UDPPraguevsCubic"
cdf_data = get_cdf_data(paths, "thrpt")
plot_cdf_matplotlib(
    data_dict=cdf_data,
    xlabel="Throughput (mbps)",
    ylabel="CDF",
    title=f"{scenario} Throughput CDF",
    filename=f"{scenario}_Thrpt_CDF",
    folder=graph_directory,
)

scenario = "PktLoss_UDPPraguevsCubic"
cdf_data = get_cdf_data(paths, "Lost_Packets")
plot_cdf_matplotlib(
    data_dict=cdf_data,
    xlabel="Packet Loss",
    ylabel="CDF",
    title=f"{scenario} Packet Loss CDF",
    filename=f"{scenario}_PktLoss_CDF",
    folder=graph_directory,
)


# Matplotlib Boxplot

In [ ]:
import pandas as pd

def combine_for_boxplot(paths, ycolumn):
    combined_rows = []
    for label, df in paths.items():
        temp_df = df[[ycolumn]].copy()
        temp_df['Scenario'] = label
        combined_rows.append(temp_df)
    combined_df = pd.concat(combined_rows, ignore_index=True)
    return combined_df


scenario = "RTT_UDPPraguevsCubic"
boxplot_df = combine_for_boxplot(paths, "SmoothedRTT")
plot_boxplot_seaborn(
    dataframe=boxplot_df,
    column="SmoothedRTT",
    groupby="Scenario",
    title=f"{scenario} SmoothedRTT Boxplot",
    xlabel="Scenario",
    ylabel="SmoothedRTT (ms)",
    filename=f"{scenario}_SmoothedRTT_Boxplot",
    folder=graph_directory,
)

scenario = "Thrpt_UDPPraguevsCubic"
boxplot_df = combine_for_boxplot(paths, "thrpt")
plot_boxplot_seaborn(
    dataframe=boxplot_df,
    column="thrpt",
    groupby="Scenario",
    title=f"{scenario} Throughput Boxplot",
    xlabel="Scenario",
    ylabel="Throughput (mbps)",
    filename=f"{scenario}_Thrpt_Boxplot",
    folder=graph_directory,
)

scenario = "PktLoss_UDPPraguevsCubic"
boxplot_df = combine_for_boxplot(paths, "Lost_Packets")
plot_boxplot_seaborn(
    dataframe=boxplot_df,
    column="Lost_Packets",
    groupby="Scenario",
    title=f"{scenario} Packet Loss Boxplot",
    xlabel="Scenario",
    ylabel="Packet Loss",
    filename=f"{scenario}_PktLoss_Boxplot",
    folder=graph_directory,
)


# Plotly

In [ ]:
scenario = "SmoothedRTT_UDPPraguevsCubic"
plot_plotly_graph(
    paths=paths,
    ycolumn="SmoothedRTT",
    title=f"{scenario} SmoothedRTT",
    xlabel="Time (s)",
    ylabel="SmoothedRTT (ms)",
    filename=f'{scenario}',
    graph_directory=graph_directory,
)

scenario = "Thrpt_UDPPraguevsCubic"
plot_plotly_graph(
    paths=paths,
    ycolumn="thrpt",
    title=f"{scenario} Throughput",
    xlabel="Time (s)",
    ylabel="Throughput (mbps)",
    filename=f'{scenario}',
    graph_directory=graph_directory,
)

scenario = "PktLoss_UDPPraguevsCubic"
plot_plotly_graph(
    paths=paths,
    ycolumn="Lost_Packets",
    title=f"{scenario} Packet Loss",
    xlabel="Time (s)",
    ylabel="Packet Loss",
    filename=f'{scenario}',
    graph_directory=graph_directory,
)


In [ ]:
# # Convert paths dict of DataFrames to dict of arrays for CDF plot
# def get_cdf_data(paths, ycolumn):
#     cdf_data = {}
#     for label, df in paths.items():
#         cdf_data[label] = df[ycolumn].dropna().values
#     return cdf_data


# scenario = "RTT_UDPPraguevsCubic"
# cdf_data = get_cdf_data(paths, "SmoothedRTT")
# plot_cdf_plotly(
#     data_dict=cdf_data,
#     xlabel="SmoothedRTT (ms)",
#     ylabel="CDF",
#     title=f"{scenario} SmoothedRTT CDF",
#     filename=f"{scenario}_SmoothedRTT_CDF",
#     folder=graph_directory,
# )

# scenario = "Thrpt_UDPPraguevsCubic"
# cdf_data = get_cdf_data(paths, "thrpt")
# plot_cdf_plotly(
#     data_dict=cdf_data,
#     xlabel="Throughput (mbps)",
#     ylabel="CDF",
#     title=f"{scenario} Throughput CDF",
#     filename=f"{scenario}_Thrpt_CDF",
#     folder=graph_directory,
# )

# scenario = "PktLoss_UDPPraguevsCubic"
# cdf_data = get_cdf_data(paths, "Lost_Packets")
# plot_cdf_plotly(
#     data_dict=cdf_data,
#     xlabel="Packet Loss",
#     ylabel="CDF",
#     title=f"{scenario} Packet Loss CDF",
#     filename=f"{scenario}_PktLoss_CDF",
#     folder=graph_directory,
# )


In [ ]:
# for folder_name, files in files_dict.items():
#     print(f"Folder: {folder_name}")
#     # for file in files:
#     #     print(f"  - {file}")

#     file_udp = files[2]
#     file_cubic = files[0]

#     # print("file_cubic",file_cubic)
#     # print("file_udp",file_udp)
#     tfcubic = extract_iperf3_data_df(file_cubic)
#     utf = extract_udp_prague_to_dataframe(file_udp)





#     # Define paths
#     paths = {
#         "Cubic": tfcubic,
#         "L4S": utf,
#         # "Bandwidth": baseline_throuhgput_df
#     }

    


#     scenario = str("RTT_UDPPraguevsCubic")
#     plot_matplotlib_graph(paths=paths,
#                     ycolumn="SmoothedRTT",
#                     title=f"{scenario} SmoothedRTT",
#                     xlabel="Time (s)",
#                     ylabel="SmoothedRTT (ms)",
#                     filename=f'{scenario}',
#                     graph_directory=graph_directory,
#     )

#     scenario = str("Thrpt_UDPPraguevsCubic")
#     plot_matplotlib_graph(paths=paths,
#                     ycolumn="thrpt",
#                     title=f"{scenario} Throuhghput",
#                     xlabel="Time (s)",
#                     ylabel="Throuhghput (mbps)",
#                     filename=f'{scenario}',
#                     graph_directory=graph_directory,
#     )


#     scenario = str("PktLoss_UDPPraguevsCubic")
#     plot_matplotlib_graph(paths=paths,
#                     ycolumn="Lost_Packets",
#                     title=f"{scenario} Packet Loss",
#                     xlabel="Time (s)",
#                     ylabel="Packet Loss",
#                     filename=f'{scenario}',
#                     graph_directory=graph_directory,
#     )

